# Usage example

An usage example of strategy based on the markowitz min variance portfolio.

Your task is to propose a particular strategy similar to the one located at the folder 'strategy'. 

In [ ]:
import pandas as pd
import quantstats as qs

## Load Data

You may organize the data the way you want. Here, we downloaded all data into the `dataset` directory in parquet format.

We also use auxiliar functions from the `data_market` directory to handle these data.

In [ ]:
from data_market.datalake import load_data

dict_data = load_data()

# Let's check the keys of the dictionary, each one a DataFrame
print(dict_data.keys())

In [ ]:
# Let's check the first DataFrame: prices of US stocks
dict_data['stocks']

## Strategy execution

Following, we test the execution of our strategy: for just a single day.

In [ ]:
# You must write all the code for your strategy entirely in the strategy directory
# Then, you should import your strategy as following
from strategy.strategy_minRisk import strategy_minRisk

In [ ]:
# Execution for a single day
# Your strategy must receive .... TODO
weights = strategy_minRisk(dict_data, t = 500, size = 20)

Your strategy must return a DataFrame with the columns: `date`, `ticker` and `weights`.

In [ ]:
# Check if the returned DataFrame has the correct format
assert all([c in weights.columns.values for c in ['date', 'ticker', 'weights']])

In [ ]:
weights

## Simulation

Following we simulate our strategy and generate the final report.

In [ ]:
from simulator.strategy_simulator import strategy_simulator

# Initialize data structures to store results
ret_port = pd.Series(dtype="float64")  # Series to store portfolio returns
weights_db = pd.DataFrame()  # DataFrame to store historical weights

# Loop through a range of time values
for t in range(500, len(dict_data['stocks'].index) - 1):

    # Use the strategy simulator to get portfolio's historical weights [weights_db]
    # and its next day returns [ret_port]
    ret_port, weights_db = strategy_simulator(path = "results/",
                                              strategy = strategy_minRisk,
                                              data_source = dict_data,
                                              t = t,
                                              ret_port = ret_port,
                                              weights_db = weights_db,
                                              size = 10)

## Report

We can use the results of the simulation, saved in the directory `results`, to generate a report of our strategy using `quantstats`.

The simulation generate two parquet files:

- [ret_port.parquet](results/ret_port.parquet): DataFrame with the return of the portfolio, for each simulated datetime
- [wegiths_db.parquet](results/weights_db.parquet): DataFrame with the weights of each stock in the portfolio, for each simulated datetime

In [ ]:
ret_port = pd.read_parquet("results/ret_port.parquet")
ret_port['date'] = pd.to_datetime(ret_port['date'], format = "%Y-%m-%d")
ret_port = ret_port.set_index("date").ret_port

In [ ]:
ret_port.head(3)

Following we generate a HTML report, comparing our strategy with the SP500.

In [ ]:
qs.reports.html(ret_port, "SPY", text_description="""
    <p>Demonstration of a simple strategy.</p>
    <p><strong>Important</strong>: Transaction costs are not taken into account.</p>""")